In [1]:
import dask.dataframe as dd

# Read CSV using Dask
data = dd.read_csv("E:/semester8/Grad2/dataset_after_oversampling/multi_classification.csv")

# Use .compute() to execute operations and get the information
data_info = data.compute().info()
print(data_info)

# Get the number of rows and columns using .compute()
num_rows, num_columns = data.compute().shape
print("Number of rows4:", num_rows)
print("Number of columns4:", num_columns)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42263771 entries, 0 to 381750
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   TP2              float64
 1   TP3              float64
 2   H1               float64
 3   DV_pressure      float64
 4   Reservoirs       float64
 5   Oil_temperature  float64
 6   Flowmeter        float64
 7   Motor_current    float64
 8   COMP             int64  
 9   DV_eletric       int64  
 10  Towers           int64  
 11  MPG              int64  
 12  LPS              int64  
 13  Pressure_switch  int64  
 14  Oil_level        int64  
 15  Caudal_impulses  int64  
 16  gpsLong          float64
 17  gpsLat           float64
 18  gpsSpeed         int64  
 19  gpsQuality       int64  
 20  status           int64  
dtypes: float64(10), int64(11)
memory usage: 6.9 GB
None
Number of rows4: 42263771
Number of columns4: 21


In [2]:
pandas_data = data.compute()
print(pandas_data.shape)

(42263771, 21)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay, accuracy_score
from sklearn.model_selection import train_test_split

In [4]:
label_column = 'status'
X = pandas_data.drop(label_column, axis=1)
y = pandas_data[label_column]

print(X)
print(y)

             TP2       TP3        H1  DV_pressure  Reservoirs  \
0      -0.012000  9.758000  9.760000       -0.028    1.576000   
1      -0.012000  9.760000  9.760000       -0.028    1.578000   
2      -0.010000  9.760000  9.760000       -0.028    1.578000   
3      -0.012000  9.756000  9.756000       -0.030    1.576000   
4      -0.012000  9.756000  9.756000       -0.030    1.578000   
...          ...       ...       ...          ...         ...   
381746 -0.008495  8.717542  8.385579       -0.034    1.468000   
381747 -0.009825  8.705572  8.362795       -0.034    1.467825   
381748 -0.009663  8.705687  8.368437       -0.034    1.467663   
381749 -0.009996  8.704039  8.360069       -0.034    1.467996   
381750 -0.009588  8.706059  8.370295       -0.034    1.467588   

        Oil_temperature  Flowmeter  Motor_current  COMP  DV_eletric  Towers  \
0             63.350000  19.049625         3.9550     1           0       1   
1             63.250000  19.049625         4.0275     1      

In [5]:
from sklearn.preprocessing import StandardScaler

# Normalize the features using StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [10]:
from keras.utils import to_categorical

# Convert target labels to one-hot encoded format
num_classes = len(np.unique(y))
y_one_hot_train = to_categorical(y_train, num_classes=num_classes)
y_one_hot_test = to_categorical(y_test, num_classes=num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)


In [11]:

# LSTM expects input data in a specific format (samples, time steps, features)
# Reshape the data for LSTM
X_train_reshaped = np.array(X_train).reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = np.array(X_test).reshape((X_test.shape[0], 1, X_test.shape[1]))


In [12]:
print(X_train_reshaped.shape)
print(X_test_reshaped.shape)

(29584639, 1, 20)
(12679132, 1, 20)


In [14]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import load_model
from keras.callbacks import EarlyStopping

# Build the LSTM+LSTM model
model = Sequential()

# First LSTM layer
model.add(LSTM(units=100, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.2))

# Second LSTM layer
model.add(LSTM(units=100, return_sequences=False))  # set return_sequences=False
model.add(Dropout(0.2))

# Output layer for multi-class classification
model.add(Dense(4, activation='softmax'))  # 4 classes, so 4 units and softmax activation

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    X_train_reshaped, y_one_hot_train,  # using one-hot encoded target labels
    epochs=100,
    batch_size=32,
    validation_data=(X_test_reshaped, y_one_hot_test),  # using one-hot encoded target labels
    callbacks=[early_stopping],
    verbose=1
)

# Save the model
model.save("LSTM_LSTM_multi.h5")

# Load the model
loaded_model = load_model("LSTM_LSTM_multi.h5")

# Evaluate the loaded model
loss, accuracy = loaded_model.evaluate(X_test_reshaped, y_one_hot_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch 1/100


924520/924520 [==============================] - 4991s 5ms/step - loss: 0.1807 - accuracy: 0.9211 - val_loss: 0.1563 - val_accuracy: 0.9324
Epoch 2/100
924520/924520 [==============================] - 3460s 4ms/step - loss: 0.1578 - accuracy: 0.9317 - val_loss: 0.1373 - val_accuracy: 0.9386
Epoch 3/100
924520/924520 [==============================] - 3167s 3ms/step - loss: 0.1493 - accuracy: 0.9351 - val_loss: 0.1419 - val_accuracy: 0.9390
Epoch 4/100
924520/924520 [==============================] - 4153s 4ms/step - loss: 0.1462 - accuracy: 0.9367 - val_loss: 0.1541 - val_accuracy: 0.9394
Epoch 5/100
924520/924520 [==============================] - 3776s 4ms/step - loss: 0.1441 - accuracy: 0.9379 - val_loss: 0.1385 - val_accuracy: 0.9415


c:\Users\ahmed mohamed\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


396223/396223 [==============================] - 1060s 3ms/step - loss: 0.1373 - accuracy: 0.9386
Test Accuracy: 93.86%


In [15]:
# Print the summary of the model
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 1, 100)            48400     
                                                                 
 dropout_4 (Dropout)         (None, 1, 100)            0         
                                                                 
 lstm_5 (LSTM)               (None, 100)               80400     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 4)                 404       
                                                                 
Total params: 129204 (504.70 KB)
Trainable params: 129204 (504.70 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
